Trying to solve a forcastign problem using a simple neural network.

# Imports

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

# Metasettings

In [ ]:
warnings.filterwarnings('ignore') # Inhibit warning in console oiutputs

# Stock Market Data Analysis

In [ ]:
# Load the dataset
dataset = 'google-stock-dataset.csv'
dataframe = pd.read_csv(dataset)
closed_stock_data = dataframe['Close'].astype(str).str.replace(',', '').astype(float)

# Splitting the data 75/25

split_index = int(len(closed_stock_data) * 0.75)
train_data = closed_stock_data[:split_index]
validation_data = closed_stock_data[split_index:]

# Data Analysis & Vizualization

In [ ]:
# Creating the DataFrame
closed_stock_data_analysis = pd.DataFrame()
closed_stock_data_analysis['Close'] = train_data

# Calculating moving averages
closed_stock_data_analysis['SMA Close:21'] = closed_stock_data.rolling(window=21).mean()
closed_stock_data_analysis['SMA Close:50'] = closed_stock_data.rolling(window=50).mean()
closed_stock_data_analysis['SMA Close:99'] = closed_stock_data.rolling(window=99).mean()

In [ ]:
# Plotting the training data
closed_stock_data_analysis[[
    'Close',
    'SMA Close:21',
    'SMA Close:50',
    'SMA Close:99'
]].plot(figsize=(16, 6))

plt.show()

In [ ]:
# Creating the DataFrame
validation_stock_data_analysis = pd.DataFrame()
validation_stock_data_analysis['Close'] = validation_data

# Calculating moving averages
validation_stock_data_analysis['SMA Close:21'] = closed_stock_data.rolling(window=21).mean()
validation_stock_data_analysis['SMA Close:50'] = closed_stock_data.rolling(window=50).mean()
validation_stock_data_analysis['SMA Close:99'] = closed_stock_data.rolling(window=99).mean()

In [ ]:
# Plotting the training data
validation_stock_data_analysis[[
    'Close',
    'SMA Close:21',
    'SMA Close:50',
    'SMA Close:99'
]].plot(figsize=(16, 6))

plt.show()

# Normalizating Training Data

In [ ]:
# Normalizing between 0 and 1
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
closed_stock_data_normalized = normalize(closed_stock_data)
closed_stock_data_normalized.plot(figsize=(16, 6))

In [ ]:
validation_data_normalized = normalize(validation_data)
validation_data_normalized.plot(figsize=(16, 6))

# Neural Network Architecture

In [ ]:
class NeuralNetwork(object):
    # Define Hyperparameters
    def __init__(self, inputLayerSize, hiddenLayerSize, outputLayerSize):
        # Architecture
        self.inputLayerSize  = inputLayerSize
        self.hiddenLayerSize = hiddenLayerSize
        self.outputLayerSize = outputLayerSize
        
        # Weights
        self.W1 = np.random.randn(self.inputLayerSize,  self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize, self.outputLayerSize)

    # Propogate inputs though network
    def forward(self, X):
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)

        yHat = self.sigmoid(self.z3) 

        return yHat

    # Apply sigmoid activation function to scalar, vector, or matrix
    def sigmoid(self, x, tetha = 0, g = 1):
        return 1 / (1 + np.exp(-g * (x - tetha)))
    
    # Gradient of sigmoid
    def sigmoidPrime(self, x, tetha = 0, g = 1):
        return np.exp(-g * (x - tetha)) / ((1 + np.exp(-g * (x - tetha))) ** 2)
    
    # The mean square error
    def costFunction(self, X, y):
        self.yHat = self.forward(X)
        return ((y - self.yHat) ** 2)

    # Const function
    def costFunctionPrime(self, X, y):
        self.yHat = self.forward(X)

        delta3 = np.multiply(- (y - self.yHat), self.sigmoidPrime(self.z3))
        dJdW2  = np.array(self.a2) * delta3

        delta2 = np.dot(delta3, self.W2.T) * self.sigmoidPrime(self.z2)
        dJdW1  = np.dot(np.array(X).reshape(5, 1), np.array(delta2).reshape(1, 10))

        return dJdW1, dJdW2

    # Helper Functions for interacting with other classes:
    # Get W1 and W2 unrolled into vector:
    def getParams(self):  
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params

    # Set W1 and W2 using single paramater vector.
    def setParams(self, params):
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))

# Forward Propagation

In [ ]:
NN = NeuralNetwork(5, 10, 1)

In [ ]:
epocs = 1

epoch_target_array = []
epoch_activity_array = []

for epoch in range(epocs):
    for index in range(len(dataframe.index) - 5):
        # Input values for the input layer
        step_input = closed_stock_data_normalized[index:index + 5]

        # Target values for the output neuron
        target = closed_stock_data_normalized[index + 5]
        epoch_target_array.append(target)

        # Calculating the activity of the step
        step_activity = NN.forward(step_input)
        epoch_activity_array.append(step_activity)

        print("Input layer: ", np.array(step_input), "\nStep target value:", target, "\nStep activity:", step_activity, "\n")

In [ ]:
forward_propagation_analysis = pd.DataFrame()

forward_propagation_analysis['Normalized Input'] = closed_stock_data_normalized
forward_propagation_analysis['Epoch Targets']    = pd.DataFrame(epoch_target_array)
forward_propagation_analysis['Epoch Activities'] = pd.DataFrame(epoch_activity_array)

forward_propagation_analysis[['Normalized Input', 'Epoch Targets', 'Epoch Activities']].plot(figsize=(16, 6))

# Backpropagation

## Activation Function

In [ ]:
def sigmoid(x, tetha = 0, g = 1):
    return 1 / (1 + np.exp(-g * (x - tetha)))

In [ ]:
def sigmoidPrime(x, tetha = 0, g = 1):
    return np.exp(-g * (x - tetha)) / ((1 + np.exp(-g * (x - tetha))) ** 2)

In [ ]:
testValues = np.arange(-5, 5, 0.01)

In [ ]:
plt.plot(testValues, sigmoid(testValues), linewidth=2)
plt.plot(testValues, sigmoidPrime(testValues), linewidth=2)
plt.grid(1)
plt.legend(['sigmoid', 'sigmoidPrime'])

# Training

In [ ]:
NN = NeuralNetwork(5, 10, 1)
learning_rate = 0.9

In [ ]:
epocs = 1000

epoch_target_array   = []
epoch_activity_array = []
squared_error_array  = []

mse_over_epochs = []

for epoch in range(epocs):
    for index in range(len(closed_stock_data_normalized) - 5):
        # Input values for the input layer
        step_input = closed_stock_data_normalized.iloc[index:index + 5].values

        # Target values for the output neuron
        target = closed_stock_data_normalized.iloc[index + 5]
        epoch_target_array.append(target)

        # Calculating the activity of the step
        step_activity = NN.forward(step_input)
        epoch_activity_array.append(step_activity)

        # Squared Error
        squared_error = NN.costFunction(step_input, target)
        squared_error_array.append(squared_error)

        # Updating weights
        dJdW1, dJdW2 = NN.costFunctionPrime(step_input, target)

        NN.W1 -= learning_rate * dJdW1
        NN.W2 -= learning_rate * dJdW2.reshape(10, 1)

    mse = np.mean(np.array(squared_error_array))
    mse_over_epochs.append(mse)
    print(f"epoch: {epoch} | mean Squared Error:", mse)

optimized_parameters = NN.getParams()

In [ ]:
back_propagation_analysis = pd.DataFrame()

back_propagation_analysis['Normalized Input'] = closed_stock_data_normalized
back_propagation_analysis['Epoch Targets']    = pd.DataFrame(epoch_target_array)
back_propagation_analysis['Prediction Plot']  = pd.DataFrame(epoch_activity_array)

back_propagation_analysis[['Normalized Input', 'Epoch Targets', 'Prediction Plot']].plot(figsize=(16, 6))

In [ ]:
plt.plot(range(epocs), mse_over_epochs)
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.title('MSE over Epochs')
plt.show()

In [ ]:
# Check the structure and length of the validation data
print("Validation data type:", type(validation_data_normalized))
print("Validation data length:", len(validation_data_normalized))

# Lists to store the results
validation_target_array = []
validation_activity_array = []
validation_squared_error_array = []

# Iterate through the validation data
for index in range(len(validation_data_normalized) - 5):
    try:
        # Input values for the input layer
        step_input = validation_data_normalized.iloc[index:index + 5].values

        # Debug print to check the indices and input values
        print(f"Index: {index}, Step Input: {step_input}")

        # Target values for the output neuron
        target = validation_data_normalized.iloc[index + 5]

        # Debug print to check the target value
        print(f"Target: {target}")

        validation_target_array.append(target)

        # Calculating the activity of the step
        step_activity = NN.forward(step_input)
        validation_activity_array.append(step_activity)

        # Squared Error
        squared_error = NN.costFunction(step_input, target)
        validation_squared_error_array.append(squared_error)

    except KeyError as e:
        print(f"KeyError at index {index}: {e}")
        break
    except Exception as e:
        print(f"Exception at index {index}: {e}")
        break

# Calculate Mean Squared Error for validation data
if validation_squared_error_array:
    validation_mse = np.mean(np.array(validation_squared_error_array))
    print("Validation Mean Squared Error:", validation_mse)
else:
    print("No validation squared errors were computed due to early stopping.")

# If you want to visualize the results
if validation_target_array and validation_activity_array:
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 5))
    plt.plot(validation_target_array, label='Actual')
    plt.plot(validation_activity_array, label='Predicted')
    plt.legend()
    plt.title('Actual vs Predicted values on Validation Data')
    plt.xlabel('Time step')
    plt.ylabel('Normalized Stock Price')
    plt.show()
else:
    print("Insufficient data for plotting.")